# Predictive Analytics for Sales Forecasting

This notebook demonstrates the use of **predictive analytics and machine learning models** to forecast retail sales using the Superstore dataset.
We apply statistical and machine learning techniques — ARIMA, Random Forest, and XGBoost — to identify seasonal patterns, build predictive models, and visualize forecast results.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from pmdarima import auto_arima

import warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.read_csv("train.csv")
df.head()
df.info()
df.describe()


In [ ]:
# Convert to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])

# Extract month and year
df['Month'] = df['Order Date'].dt.month
df['Year'] = df['Order Date'].dt.year

# Group by Category for lag and moving average
df = df.sort_values(['Category', 'Order Date'])
df['Sales Lag1'] = df.groupby(['Category'])['Sales'].shift(1)
df['Sales MA3'] = df.groupby(['Category'])['Sales'].transform(lambda x: x.rolling(3).mean())

df = df.dropna(subset=['Sales Lag1', 'Sales MA3'])
df.head()


In [ ]:
# Prepare data for modeling
features = ['Sales Lag1', 'Sales MA3']
X = df[features]
y = df['Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)



In [ ]:
# Model 1 ARIMA (Classical Forecasting)


# Aggregate monthly sales
df_monthly = df.groupby(pd.Grouper(key='Order Date', freq='M'))['Sales'].sum()

train_size = int(len(df_monthly) * 0.8)
train_series = df_monthly[:train_size]
test_series = df_monthly[train_size:]

model_arima = auto_arima(train_series, seasonal=False, trace=True)
forecast_arima = model_arima.predict(n_periods=len(test_series))

plt.figure(figsize=(10,5))
plt.plot(train_series.index, train_series, label='Train')
plt.plot(test_series.index, test_series, label='Test')
plt.plot(test_series.index, forecast_arima, label='ARIMA Forecast')
plt.legend()
plt.title('ARIMA Forecast vs Actual Sales')
plt.show()


In [ ]:
# Model 2 Random Forest Regressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

print("Random Forest RMSE:", np.sqrt(mean_squared_error(y_test, rf_pred)))
print("Random Forest R²:", r2_score(y_test, rf_pred))


In [ ]:
# Model 3 XGBoost Regressor

xgb_model = XGBRegressor(n_estimators=200, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

print("XGBoost RMSE:", np.sqrt(mean_squared_error(y_test, xgb_pred)))
print("XGBoost R²:", r2_score(y_test, xgb_pred))


In [ ]:
# Compare Model Performance

results = pd.DataFrame({
    'Model': ['ARIMA', 'Random Forest', 'XGBoost'],
    'RMSE': [
        np.sqrt(mean_squared_error(test_series, forecast_arima)),
        np.sqrt(mean_squared_error(y_test, rf_pred)),
        np.sqrt(mean_squared_error(y_test, xgb_pred))
    ]
})
results


In [ ]:
#Visualize Actual vs Predicted (XGBoost)

plt.figure(figsize=(10,5))
plt.plot(y_test.values, label='Actual Sales', marker='o')
plt.plot(xgb_pred, label='Predicted Sales (XGBoost)', marker='x')
plt.title('Actual vs Predicted Sales')
plt.xlabel('Index')
plt.ylabel('Sales')
plt.legend()
plt.grid(True)
plt.show()


### Conclusion
Among the models tested, **XGBoost** provided the most accurate and consistent results, outperforming ARIMA and Random Forest in RMSE and R² metrics.
The study demonstrates that predictive analytics, combined with machine learning, can significantly enhance sales forecasting accuracy and business planning efficiency.
